In [2]:
import pandas as pd
import os

In [3]:
os.chdir('/home/malachy/ADS (2024)/project-2-group-real-estate-industry-project-11/notebooks')

## Functions

### Preparing suburb name

In [4]:
import numpy as np

DIRECTIONS = ["north", "south", "west", "east"]

"""`df` should have unique indicies and"""
def suburb_name_split(df):
    indicies = df.index.values
    final_indicies = []
    repeated_indicies = []

    # for each index, gather how many indicies to repeat and the final product
    for index in indicies:
        # remove (.vic) if present
        clean_index = index.replace("(Vic.)", "")

        # replace the brackets around "west"
        clean_index = clean_index.replace("(", "").replace(")", "")

        # make sure lower case, no confusion
        clean_index = clean_index.lower()

        # nothing fancy
        if (not "-" in clean_index):
            final_indicies.append(clean_index)
            repeated_indicies.append(index)
            continue
        
        # need to split
        index_split = clean_index.split(" - ")

        if (len(index_split) != 2):
            print("length of index split is not 2?")
            exit()
        
        # if only one direction, reintroduce without the weird thing
        if (index_split[1].lower() in DIRECTIONS):
            final_indicies.append(" ".join(index_split))
            repeated_indicies.append(index)
        
        # else will need to strip everything
        else:
            final_indicies.extend(index_split)
            repeated_indicies.extend(np.repeat(index, 2))
    
    # duplicate
    df = df.loc[repeated_indicies]

    # change the names accordingly
    df.index = final_indicies
    
    return df


### Column and imputation

In [5]:
def impute_previous(columns, na):
    flat_column_list = [[group[i] for group in columns] for i in range(len(columns[0]))]

    column_dataframe = pd.DataFrame(flat_column_list)

    column_dataframe = column_dataframe.applymap(lambda x: None if (not x) or (na in str(x)) else x)

    column_dataframe = column_dataframe.T.ffill().T

    flat_column_list = column_dataframe.to_numpy()

    return flat_column_list

In [6]:
def fill_columns(df, column_dict, na="Unnamed"):
    columns = list(df.columns)

    for index, new_column in column_dict.items():
        columns[index] = new_column

    if (type(df.columns) == pd.MultiIndex):
        flat_column_list = impute_previous(columns, na)

        df.columns = pd.MultiIndex.from_arrays(flat_column_list)
    
    else:
        df.columns = columns

    return df

## Economic by Region

In [7]:
def filter_victoria(df, column_name):
    region_mask = ~df[column_name].str.isnumeric()

    # get the region indexes and names
    regions_index, regions_names = list(df[region_mask].index), list(df[region_mask][column_name].values)

    # find the index for victoria in the list
    index_victoria = regions_names.index("Victoria")

    # reference the mask between victoria and the next index
    df = df.loc[regions_index[index_victoria] + 1: regions_index[index_victoria + 1] - 1]

    return df

### Geography summary

In [8]:
COLUMNS_DICT_INCOME_GEOGRAPHY = {
    0: ("SA2 code",None),
    1: ("SA2 name",None)
}

In [9]:
df = pd.read_csv("../data/landing/economic_by_region/income_by_geography_b2022.csv", header=[0, 1])

df = fill_columns(df, COLUMNS_DICT_INCOME_GEOGRAPHY)

df = filter_victoria(df, ("SA2 code", None))

In [10]:
df = df.melt(id_vars=df.columns.to_list()[:2], 
             value_vars = df.columns.to_list()[2:],
             var_name=["Measure", "Year"],
             value_name="Value")

In [11]:
new_df = df.pivot(index=list(df.columns[:2]) + ["Year"], columns="Measure", values="Value")
new_df = new_df.reset_index()
new_df.columns = [x[0] for x in new_df.columns[:2]] + list(new_df.columns[2:])
#new_df = new_df.set_index("SA2 name")
new_df

,SA2 code,SA2 name,Year,Earners (persons),Mean ($),Median ($),Median age of earners (years),Sum ($)
0,201011001,Alfredton,2016-17,"7,117","60,937","50,596",42,"433,690,088"
1,201011001,Alfredton,2017-18,"7,558","62,343","52,448",42,"471,188,331"
2,201011001,Alfredton,2018-19,"7,987","63,731","53,932",42,"509,017,733"
3,201011001,Alfredton,2019-20,"8,665","65,781","55,204",41,"569,994,001"
4,201011001,Alfredton,2020-21,"9,438","69,111","58,036",41,"652,268,973"
...,...,...,...,...,...,...,...,...
2605,217041480,Warrnambool - South,2016-17,"7,614","54,442","43,149",44,"414,520,667"
2606,217041480,Warrnambool - South,2017-18,"7,737","54,826","45,726",44,"424,190,279"
2607,217041480,Warrnambool - South,2018-19,"8,051","58,145","47,242",44,"468,127,252"
2608,217041480,Warrnambool - South,2019-20,"8,096","58,648","49,080",44,"474,817,395"


In [12]:
new_df.index.values[:20]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [13]:
new_df = suburb_name_split(new_df)
new_df

AttributeError: 'numpy.int64' object has no attribute 'replace'

### Distribution

In [12]:
COLUMNS_DICT_INCOME_DISTRIBUTION = {
    0: "SA2 code",
    1: "SA2 name"
}

In [13]:
df = pd.read_csv("../data/landing/economic_by_region/income_distribution_by_geography_2021.csv", header=0)
df = df.drop(0)

df = fill_columns(df, COLUMNS_DICT_INCOME_DISTRIBUTION)

df = filter_victoria(df, "SA2 code")

df = df.set_index("SA2 name")

df = suburb_name_split(df)

## Economic

- Come back to

## Housing